# Recommender System for E-Commerce

This notebook is part of an internship project under YBI Foundation.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Sample dataset of users and their purchased products
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6],
    'product': ['Laptop', 'Mouse', 'Laptop', 'Keyboard', 'Mouse', 'Laptop', 'Monitor',
                'Keyboard', 'Headphones', 'Laptop', 'Monitor', 'Mouse']
}
df = pd.DataFrame(data)
print("User Purchase History:")
print(df)

In [ ]:
# Creating User-Item Matrix
user_item_matrix = pd.crosstab(df['user_id'], df['product'])
print("\nUser-Item Matrix:")
print(user_item_matrix)

In [ ]:
# Visualizing user-product interaction using heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(user_item_matrix, annot=True, cmap='YlGnBu')
plt.title("User vs Product Interaction")
plt.xlabel("Products")
plt.ylabel("Users")
plt.show()

In [ ]:
# Calculating User Similarity Matrix
user_similarity = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
print("\nUser Similarity Matrix:")
print(user_sim_df.round(2))

In [ ]:
# Calculating Item Similarity Matrix
item_matrix = user_item_matrix.T
item_similarity = cosine_similarity(item_matrix)
item_sim_df = pd.DataFrame(item_similarity, index=item_matrix.index, columns=item_matrix.index)
print("\nItem Similarity Matrix:")
print(item_sim_df.round(2))

In [ ]:
# Function for User-Based Recommendation
def recommend_user_based(user_id, top_n=3):
    if user_id not in user_sim_df.index:
        return []
    sim_users = user_sim_df[user_id].drop(user_id).sort_values(ascending=False)
    user_products = set(df[df['user_id'] == user_id]['product'])
    recommendations = {}
    for sim_user_id, sim_score in sim_users.items():
        products = df[df['user_id'] == sim_user_id]['product']
        for product in products:
            if product not in user_products:
                recommendations[product] = recommendations.get(product, 0) + sim_score
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return [product for product, _ in sorted_recommendations][:top_n]

In [ ]:
# Function for Item-Based Recommendation
def recommend_item_based(target_product, top_n=3):
    if target_product not in item_sim_df.columns:
        return []
    sim_scores = item_sim_df[target_product].drop(target_product).sort_values(ascending=False)
    return sim_scores.head(top_n).index.tolist()

In [ ]:
# Running user-based recommendations
print("\nUser-Based Recommendations:")
for uid in user_item_matrix.index:
    recs = recommend_user_based(uid)
    print(f"User {uid}: {recs}")

# Running item-based recommendation example
print("\nItem-Based Recommendations (related to 'Laptop'):")
print(recommend_item_based('Laptop'))

# Ending statement
print("\nThis notebook demonstrates both user- and item-based collaborative filtering with a simulated dataset.")